# Classes

In [1]:
class Vehicle:
    """example class"""
    def __init__(self, wheels, seats):
        """
        The __init__ method is used for object creation.
        """
        self.wheels = wheels
        self.seats = seats
    def drive(self, km):
        print("driving {} km on {} wheels".format(km, self.wheels))

In [2]:
bicycle = Vehicle(2, 1)
bicycle.drive(10)

driving 10 km on 2 wheels


In [3]:
bicycle.wheels

2

## Inheritance

In [4]:
class Car(Vehicle):
    def __init__(self, seats, power):
        super().__init__(wheels=4, seats=seats) # calls constructuor of parent class
        self.power = power
    def drive(self, km):
        print("driving {} km on {} wheels with {} kW power".format(
            km, self.wheels, self.power))

In [5]:
my_car = Car(5, 150)
my_car.drive(200)

driving 200 km on 4 wheels with 150 kW power


If a method is overwritten in the child class (like init in the example), the method of the parent class can be accessed with super().

### Multiple Inheritance

In [6]:
class PurchaseMixin:
    price = None
    currency = 'EUR'
    nr_owned = 0
    def buy(self, nr=1):
        self.nr_owned += nr
        print("purchased {} piece(s) for {} {}".format(nr, nr*self.price, self.currency))
    def sell(self, nr=1):
        if nr > self.nr_owned:
            raise ValueError('you own less than you want to sell!')
        self.nr_owned -= nr
        print("sold {} piece(s) for {} {}".format(nr, nr*self.price, self.currency))

In [7]:
class PurchasableCar(PurchaseMixin, Car):
    pass

In [8]:
my_cars = PurchasableCar(5, 150)
my_cars.price = 30000
my_cars.buy(2)
my_cars.sell()
my_cars.drive(250)
my_cars.nr_owned

purchased 2 piece(s) for 60000 EUR
sold 1 piece(s) for 30000 EUR
driving 250 km on 4 wheels with 150 kW power


1

Classes in Python may inherit from multiple parent classes. This can be used to add additional features to classes.

### Naming Resolution

In [9]:
class A:
    def __init__(self):
        self.i = 1
        print('init A')
    def func(self, x):
        return x+1
class B:
    def __init__(self):
        self.i = 2
        print('init B')
    def func(self, x):
        return x+2

In [10]:
class AB(A,B):
    pass
class BA(B,A):
    pass

In [11]:
ab = AB()
ba = BA()
ab.func(0), ba.func(0)

init A
init B


(1, 2)

If parent classes have the same methods (including init), the derived class uses the method of the 1st parent.

In [12]:
class AB2(A, B):
    def func(self,x):
        return B.func(self, x)

In [13]:
ab2 = AB2()
ab2.func(0)

init A


2

Methods in the 2nd parent class, which have been overwritten my identically named methods in the 1st parent class, can be called using the parent class name as prefix.

## Property Methods

In [14]:
my_cars.price = 'hello'
my_cars.buy() # output is not what is expected

purchased 1 piece(s) for hello EUR


In languages like C++ and Java one usually implements getter and setter methods for any public property. 
This is discouraged in Python because it creates a lot of boilerplate code.
Instead, properties should be accessed directly.

If it turns out later that logic (like validations, formatting) is required when getting or setting properties, they can be changed in the class definition to property methods without changing the class api.

In [15]:
from numbers import Number
class SafePurchaseMixin(PurchaseMixin):
    _price = None
    
    @property
    def price(self):
        if self._price is None:
            raise ValueError('price not set')
        return self._price
    
    @price.setter
    def price(self, x):
        if not isinstance(x, Number):
            raise TypeError('Price must be numeric')
        if x <= 0:
            raise ValueError('Price must be positive')
        self._price = x
        
    @price.deleter # for completeness, often not required
    def price(self):
        self._price = None

In [16]:
class PurchasableCar2(SafePurchaseMixin, Car):
    pass

In [17]:
my_cars = PurchasableCar2(5, 150)
my_cars.price = 30000 # note that the api is identical to PurchasableCars
my_cars.buy(2)
my_cars.sell()
my_cars.drive(250)
my_cars.nr_owned

purchased 2 piece(s) for 60000 EUR
sold 1 piece(s) for 30000 EUR
driving 250 km on 4 wheels with 150 kW power


1

In [18]:
try:
    my_cars.price = 'hello'
except TypeError as e:
    print(e)

Price must be numeric


In [19]:
del my_cars.price
try:
    print(my_cars.price)
except ValueError as e:
    print(e)

price not set


In the property methods for price, validation has been included without any change of the class api.

## Static Methods

In [20]:
class PurchasableCar3(PurchasableCar2):
    @staticmethod
    def convert_kw_to_hp(kw):
        return 1.36 * kw

In [21]:
PurchasableCar3.convert_kw_to_hp(120)

163.20000000000002

Methods inside a class which do not need any input from the class itself, can (and should) be defined as static methods. This is done as follows:

1. the decorator @staticmethod
2. self is not a parameter of the method

Static mehods can be used analogue to standard object methods. In addition, it they could be called on the class itself, without need to instanciate an object.

# Class Methods

In [22]:
class PurchasableCar4(PurchasableCar3):
    @classmethod
    def from_dict(cls, data):
        obj = cls(seats=data.get('seats'),
                 power=data.get('power'))
        obj.price=data.get('price')
        obj.currency=data.get('currency','EUR')
        return obj

In [23]:
car_data = {'seats': 2, 'power': 250, 'price': 125000}
sports_car = PurchasableCar4.from_dict(car_data)
print(f'''seats: {sports_car.seats}, 
power in hp: {sports_car.convert_kw_to_hp(sports_car.power)},
price: {sports_car.price} {sports_car.currency}''')

seats: 2, 
power in hp: 340.0,
price: 125000 EUR


Class methods are executed on classes, not objects (i.e. instances of classes).
A common use case for class methods are alternative constructors which return class instances.

More about static and class methods here:

https://realpython.com/instance-class-and-static-methods-demystified/

## Dunder Methods

In [24]:
from math import sqrt
class ComplexNumber:
    """
    just as an example, in practice use the built-in complex data type!
    """
    def __init__(self,real, imag=0):
        self.r = real
        self.i = imag
    
    @staticmethod
    def _convert_num(x):
        """
        Converts number or ComplexNumber into real and imaginary part.
        This is an internal method, marked by leading underscore (_).
        """
        if isinstance(x, ComplexNumber):
            r2 = x.r
            i2 = x.i
        else:
            r2 = x
            i2 = 0
        return r2, i2
    
    def __str__(self):
        """Defines string representation of the object."""
        return "{r} + {i} * i".format(r=self.r, i=self.i)
    
    def __repr__(self):
        """Defines a printable representation of the object. 
        Usually __repr__ contains more debugging information than __str__."""
        return "ComplexNumber object, value: {r} + {i} * i".format(r=self.r, i=self.i)
    
    def __add__(self, x):
        r2, i2 = self._convert_num(x)
        return ComplexNumber(self.r + r2, self.i + i2)

    def get_abs_value(self):
        return sqrt(self.r**2 + self.i**2)

In [25]:
c1 = ComplexNumber(1, 2)
print(c1)

1 + 2 * i


The print function implicitly converts the ComplexNumber object into a string, i.e. printing the output of the \_\_str__ method.

In [26]:
c1

ComplexNumber object, value: 1 + 2 * i

In [27]:
c1.r, c1.i, c1.get_abs_value()

(1, 2, 2.23606797749979)

In [28]:
c2 = ComplexNumber(2, 3)
c3 = c1 + c2
print(c3)

3 + 5 * i


In [29]:
print(c1 + 7)

8 + 2 * i


## Singleton Pattern

A singleton is a class which may have only one instance. Any attempt to create a second instance of this class instead returns a pointer to the existing instance.

Singleton support is not included in Python standard library. Indeed, it is very controversial if singletons should be used in Python at all, many consider this as an anti-pattern because of its intransparency and global state.

There are many possibilities to introduce singletons in Python, the metaclass method below is probably the best.

Source: https://stackoverflow.com/questions/6760685/creating-a-singleton-in-python

In [30]:
class Singleton(type):
    _instances = {}
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(Singleton, cls).__call__(*args, **kwargs)
        return cls._instances[cls]

In [31]:
class MySingleton(metaclass=Singleton):
    def __init__(self):
        print('singleton initiated')

In [32]:
s1 = MySingleton()
s1.x = 'hello world'
hex(id(s1))

singleton initiated


'0x7fe0d42cf390'

In [33]:
s2 = MySingleton()
hex(id(s2))

'0x7fe0d42cf390'

In [34]:
s2.x

'hello world'

# Namespaces

In [35]:
outside1 = 42
outside2 = 0
def f_bad(x):
    outside2 = x # this is actually a local variable 
                 # which has nothing to do with outside 2 defined before
    print(outside2)
    return x + outside1 # extremely dangerous usage of global variable

In [36]:
outside2

0

In [37]:
f_bad(7), outside2

7


(49, 0)

Variables defined inside a function (or class) are local, i.e. they are only valid inside the function and changes of them do not influence anything outside the function. This is true even if outside variables have the same name (here global2).

Variables defined outside the function, however, can be accessed from inside the function. This is however very dangerous and should be avoided!

In [38]:
def f_even_worse(x):
    global outside2 # makes the variable defined outside changable inside the function
    outside2 = x # the outside variable is actually changed here!
    print(outside2)
    return x + outside1 # extremely dangerous usage of global variable

In [39]:
outside2

0

In [40]:
f_even_worse(7), outside2

7


(49, 7)

The *global* stament makes the variable defined outside changable inside the function. The usage of this pattern is __extremely__ dangerous and must be avoided!

In [41]:
def f_good(x, i):
    print(x)
    return x + i

In [42]:
f_good(7, 42)

7


49

Refactored version of the previous function without implicit usage of variable defined outside the function.

In [43]:
def f_mod_input(i: int, l: list):
    l.append(i)
    i = i*2
    return i

In [44]:
i1 = 42 # basic variable
l1 = [1, 2] # list object
f_mod_input(i1, l1)

84

In [45]:
i1, l1

(42, [1, 2, 42])

Both function parameters (here integer and list) are changed inside the function.

* The basic type (here integer) is however not changed in the outside scope - call by value
* The complex type (here list) is changed in the outside scope by the modification inside the function - call by reference

It is a common source of errors and (in my opinion) a major weakness of Python language that call by value and call by reference are not clearly separated by syntax (like in C (++) where pointers have * as prefix).

Call by reference can be used on purpose to modify objects in outer scope by the function. However, it is safer to explicitly create a new object inside the function, modify that and return it.

In [46]:
def f_safer(i: int, l: list):
    l2 = l.copy() # creates new list object
    l2.append(i)
    i = i*2
    return i, l2

In [47]:
i1 = 42 # basic variable
l1 = [1, 2] # list object
f_safer(i1, l1)

(84, [1, 2, 42])

In [48]:
i1, l1

(42, [1, 2])

# Context Managers
## Motivation and Usage

In [49]:
with open('temp.txt','w') as f:
    f.write('hello world\n')
    f.write('test file for context manager\n')

In [50]:
with open('temp.txt','r') as f:
    content = f.readlines()
content

['hello world\n', 'test file for context manager\n']

The *with* statement is a context manager which gives access to the specified resource (here a text file) in the following code block.

The main advantage is that the context manager automatically takes care of closing the resource when the code block is finished or there is an error.

In [51]:
try:
    f = open('temp.txt')
    content = f.readlines()
finally:
    f.close()
content

['hello world\n', 'test file for context manager\n']

Implementation without context manager would be significantly more complex (here 5 lines instead of 2 for the read part) and error-prone (e.g. if it is forgotten to implement the close command or if it is not in a finally block).

## Implementing Own Context Managers
https://stackabuse.com/python-context-managers/

In [52]:
class MyFileHandler:
    """only for illustration - redundant because Python's open already includes a 
    context manager"""
    def __init__(self, filename, kind='r'):
        self.filename = filename
        self.kind = kind
    def __enter__(self):
        self.file = open(self.filename, self.kind)
        print('file opened')
        return self.file
    def __exit__(self, *exc):
        self.file.close()
        print('file closed')

In [54]:
try:
    with MyFileHandler('temp.txt','r') as f:
        content = f.readlines()
        print('file read')
        raise Exception('an unexpected problem - file should nevertheless be closed!')
except Exception as e:
    print(e)

file opened
file read
file closed
an unexpected problem - file should nevertheless be closed!


Context managers are implemented by defining the dunder methods

    __enter__(self)
    __exit__(self, *exc)
    
Note that in the example the file is closed even though an exception has been raised.

## Cleanup

In [55]:
from os import remove
remove('temp.txt')

Author: Benjamin Lungwitz